In [6]:
import glob
from tqdm.auto import tqdm

from pymatgen.io.vasp import Outcar, Poscar
import numpy as np

from ase import Atoms, units
from ase.io import write, read
from ase.calculators.singlepoint import SinglePointCalculator

In [51]:

def outcar2traj(symbols: list[str] | None, outcar: Outcar, log: bool = False) -> list[Atoms]:

    def str2floats(s: str):
        return [float(x) for x in s.split()]
    
    # energy

    row_pattern = r'energy without entropy\s*=\s*(-?\d+\.\d+)\s+energy\(sigma-\>0\)\s*=\s*(-?\d+\.\d+)'

    data = outcar.read_table_pattern(
        header_pattern=r'\s*',
        row_pattern=row_pattern,
        footer_pattern=r'\s*',
        postprocess=str2floats,
        last_one_only=False,
        first_one_only=False,
    )

    data = np.array(data).squeeze()

    outcar.data['energy'] = data[:, -1]

    # positions and forces

    header_pattern = r'^POSITION\s*TOTAL-FORCE \(eV\/Angst\)\s*\n\s*-+\s*'
    row_pattern = r'\s*(-?\d+\.\d+\s+-?\d+\.\d+\s+-?\d+\.\d+\s+-?\d+\.\d+\s+-?\d+\.\d+\s+-?\d+\.\d+)\s*'

    data = outcar.read_table_pattern(
        header_pattern=r'\s*-+\s*',
        row_pattern=row_pattern,
        footer_pattern=r'\s*-+\s*',
        postprocess=str2floats,
        last_one_only=False,
        first_one_only=False,
    )
    data = np.array(data).squeeze()

    outcar.data['positions'] = data[:, :, :3]
    outcar.data['forces'] = data[:, :, 3:]

    # lattice

    header_pattern = r'\s*direct lattice vectors\s*reciprocal lattice vectors'
    row_pattern = r'\s*(-?\d+\.\d+)\s*(-?\d+\.\d+)\s*(-?\d+\.\d+)\s*(-?\d+\.\d+)\s*(-?\d+\.\d+)\s*(-?\d+\.\d+)\s*'

    data = outcar.read_table_pattern(
        header_pattern=header_pattern, # r'\s*-+\s*',
        row_pattern=row_pattern,
        footer_pattern=r'\s*',
        postprocess=str2floats,
        last_one_only=False,
        first_one_only=False,
    )
    data = np.array(data).squeeze()

    outcar.data['lattice'] = data[:, :, :3]

    pattern_stress = r'in kB\s+(-?\d+\.\d+)\s+(-?\d+\.\d+)\s+(-?\d+\.\d+)\s+(-?\d+\.\d+)\s+(-?\d+\.\d+)\s+(-?\d+\.\d+)'

    outcar.read_pattern({
        "stress": pattern_stress,
    })

    outcar.data['stress'] = np.array(outcar.data['stress'], dtype=np.float32) * -1e-1 * units.GPa

    nsteps = min(outcar.data['positions'].shape[0], outcar.data['energy'].shape[0], outcar.data['forces'].shape[0], outcar.data['stress'].shape[0], outcar.data['lattice'].shape[0])

    outcar.data['positions'] = outcar.data['positions'][:nsteps]
    outcar.data['energy'] = outcar.data['energy'][:nsteps]
    outcar.data['forces'] = outcar.data['forces'][:nsteps]
    outcar.data['stress'] = outcar.data['stress'][:nsteps]
    outcar.data['lattice'] = outcar.data['lattice'][:nsteps]

    if log:
        print('nsteps', nsteps)
        print('energy', outcar.data['energy'].shape)
        print('positions', outcar.data['positions'].shape)
        print('forces', outcar.data['forces'].shape)
        print('stress', outcar.data['stress'].shape)
        print('lattice', outcar.data['lattice'].shape)

    traj = []
    
    # for positions, forces, stress in zip(outcar.data['positions'], outcar.data['forces'], outcar.data['stress']):
    for frame in zip(outcar.data['positions'],
                     outcar.data['energy'],
                     outcar.data['forces'], 
                     outcar.data['stress'],
                     outcar.data['lattice']):
        
        positions, energy, forces, stress, lattice = frame

        atoms = Atoms(
            symbols=symbols,
            positions=positions, 
            cell=lattice,
            pbc=True,
        )

        calc = SinglePointCalculator(atoms, energy=energy, forces=forces, stress=stress)
        atoms.calc = calc

        traj.append(atoms)

    return traj

In [52]:
outcar = Outcar('OUTCAR.gz')

traj = outcar2traj(None, outcar, log=True)

nsteps 9
energy (9,)
positions (9, 72, 3)
forces (9, 72, 3)
stress (9, 6)
lattice (9, 3, 3)


In [54]:
traj[0]

Atoms(symbols='X72', pbc=True, cell=[[25.427844466, -6.896715133, -4.77193617], [0.0, 13.530737977, 11.616758458], [-0.0, -0.0, 14.069482811]], calculator=SinglePointCalculator(...))